In [ ]:
def WriteOutput(Model, OrderSize, Results):
    Obj = pyo.value(Model.Obj())
    Products = '['
    for c in Model.Candidate:   # Collate list of selected product sizes
        if np.isclose(pyo.value(Model.Select[c]), 1):   # Binary variable = 1, give-or-take small precision error
            Products += str(pyo.value(Model.CandidateWidth[c])).rjust(6) + ' ' + str(pyo.value(Model.CandidateLength[c])).rjust(6) + ' '
    Products += ']'
    print()
    print(f'Order size:   {OrderSize:<,.0f}')
    print(f'Objective:    {Obj:<,.0f} ({Obj / pyo.value(Model.Baseline):.1%} of baseline)')
    print(f'Products:     {Products}\n')

    pd.options.display.float_format = '{:,.0f}'.format
    ItemsAllocated = pd.DataFrame()
    
    SelectedSizes = []   # Dataframe of solution
    for c in Model.Select:
        if np.isclose(pyo.value(Model.Select[c]), 1):
            SelectedSizes.append(str(pyo.value(Model.CandidateWidth[c])) + 'x' + str(pyo.value(Model.CandidateLength[c])))
            for i in Model.Item:
                ItemsAllocated.at[i, c] = pyo.value(Model.Allocation[i, c])
    ItemsAllocated.columns = SelectedSizes

    ItemSizes = []   # Add item sizes to solution dataframe
    for i in Model.Item:
        ItemSizes.append(str(pyo.value(Model.Width[i])) + 'x' + str(pyo.value(Model.Length[i])))
    ItemsAllocated['Item'] = ItemSizes
    pd.set_option('display.max_rows', None)
    display(ItemsAllocated)